In [1]:
import random
from pig_logger import logger
import pig
import importlib

importlib.reload(pig)

In [2]:
p1 = pig.RollNTimesPlayer(2)
p2 = pig.HoldAtNPlayer(6)

g = pig.PigGame(p1, p2, [3, 4])

In [3]:
g.other_player.roll()

2025-02-07 20:40:47 | INFO | pig.py:16 | RollNTimesPlayer(n_rolls=2) is rolling
2025-02-07 20:40:47 | INFO | pig.py:19 | RollNTimesPlayer(n_rolls=2) rolled a 5
2025-02-07 20:40:47 | INFO | pig.py:26 | RollNTimesPlayer(n_rolls=2) is rolling again


10

In [51]:
print([x(4) for x in PigPlayer.__subclasses__()])

TypeError: PigPlayer.__init__() takes 1 positional argument but 2 were given